# Your first quantum game

We are going to make a game, so we'll need a game engine. We'll use a simple one made out of Jupyter notebooks.

In [ ]:
from qiskit_textbook.games.jupyter_widget_engine import JupyterWidgetEngine

If the command above doesn't work, run the following cell for a hacky workaround.

In [ ]:
exec("\nfrom ipywidgets import widgets \nfrom ipywidgets import Layout, HBox, VBox\nfrom IPython.display import display\n\nclass Pixel():\n    \n    def __init__(self, layout):\n        self._button = widgets.ToggleButton(description='',button_style='',layout=layout,disabled=True)\n        \n    def set_color(self,color):\n        if color in ['grey','gray']:\n            self._button.button_style = ''\n        elif color=='green':\n            self._button.button_style = 'success'\n        elif color=='blue':\n            self._button.button_style = 'info'\n        elif color=='orange':\n            self._button.button_style = 'warning'\n        elif color=='red':\n            self._button.button_style = 'danger'\n            \n    def set_brightness(self,bright):\n        self._button.value = not bright\n        \n    def set_text(self,text):\n        self._button.description = text\n        \n\nclass JupyterWidgetEngine():\n    \n    def __init__(self,start,next_frame,L=8):\n        \n        self.next_frame = next_frame\n        self.L = L\n        \n        width = int(50*8/L)\n        wide = str(7*width+24)+'px'\n        wider = str(L*width+(L-1)*4)+'px'\n        width = str(width)+'px'\n        height = width\n        width = str(int(50*8/L))+'px'\n\n        layout = Layout(width=width, height=height)\n\n        screen = {}\n        for x in range(L):\n            for y in range(L):\n                screen[x,y] = Pixel(layout)\n        screen['text'] = Pixel(Layout(width=wider, height=height))\n\n        controller = {}\n        controller['blank'] = widgets.ToggleButton(description='',button_style='',layout=layout)\n        controller['up'] = widgets.ToggleButton(description='▲',button_style='',layout=layout)\n        controller['down'] = widgets.ToggleButton(description='▼',button_style='',layout=layout)\n        controller['left'] = widgets.ToggleButton(description='◀︎',button_style='',layout=layout)\n        controller['right'] = widgets.ToggleButton(description='►',button_style='',layout=layout)\n        controller['A'] = widgets.ToggleButton(description='A',button_style='',layout=layout)\n        controller['B'] = widgets.ToggleButton(description='B',button_style='',layout=layout)\n        controller['X'] = widgets.ToggleButton(description='X',button_style='',layout=layout)\n        controller['Y'] = widgets.ToggleButton(description='Y',button_style='',layout=layout)\n        controller['next'] = widgets.ToggleButton(description='Next',button_style='',layout=Layout(width=wide, height=height))\n\n        [b,u,d,l,r,A,B,X,Y,c] = [controller['blank'],\n                             controller['up'],\n                             controller['down'],\n                             controller['left'],\n                             controller['right'],\n                             controller['A'],\n                             controller['B'],\n                             controller['X'],\n                             controller['Y'],\n                             controller['next']]\n\n\n        interface = []\n        interface.append( widgets.HBox([screen[x,0]._button for x in range(L)]+[b,u,b,b,b,X,b]) )\n        interface.append( widgets.HBox([screen[x,1]._button for x in range(L)]+[l,b,r,b,Y,b,A]) )\n        interface.append( widgets.HBox([screen[x,2]._button for x in range(L)]+[b,d,b,b,b,B,b]) )\n        interface.append( widgets.HBox([screen[x,3]._button for x in range(L)]+[c]) )\n        for y in range(4,L):\n            interface.append( widgets.HBox([screen[x,y]._button for x in range(L)]) )\n        interface.append( screen['text']._button )\n            \n        self.screen = screen\n        self.controller = controller\n            \n        start(self)\n            \n        display(widgets.VBox(interface))\n        \n        b.observe(self.given_blank)\n        \n        for button in self.controller:\n            if button!='blank':\n                self.controller[button].observe(self.given_button)\n        \n        \n\n    def given_blank(self,obs_b):\n        if self.controller['blank'].value:\n            self.controller['blank'].value = False\n    \n\n\n    def given_button(self,obs_n):\n\n        for button in self.controller:\n            if self.controller[button].value is True:\n                self.next_frame(self)\n\n        for button in self.controller.values():\n            button.value = False\n            \n")

A minimal example of using this game engine is to simply set all the pixels to a certan colour.

In [ ]:
# function called when setting up
def start(engine):
    # just move on to the first frame
    next_frame(engine)

# this is the function that does everything
def next_frame (engine):
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
    
# run the game for an 8x8 screen
engine = JupyterWidgetEngine(start,next_frame,L=8)

Now let's set one specific pixel to a different colour.

In [ ]:
# function called when setting up
def start(engine):
    
    # set a parameter to keep track of the player pixel
    engine.X = 1
    engine.Y = 2
    
    # then move on to the first frame
    next_frame(engine)

# this is the function that does everything
def next_frame (engine):
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
            
    # draw the player pixel
    engine.screen[engine.X,engine.Y].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

We'll move this around using the arrow buttons.

In [ ]:
# this is the function that does everything
def next_frame (engine):
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
            
    # draw the player pixel
    engine.screen[engine.X,engine.Y].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

Walking off the edge of the screen results in an error. We can fix this.

In [ ]:
# this is the function that does everything
def next_frame (engine):
    
    # remove initial text
    if engine.controller['A'].value:
        engine.screen['text'].set_text('')
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            engine.screen[x,y].set_color('green')
            
    # draw the player pixel
    Xs = engine.X%engine.L
    Ys = engine.Y%engine.L
    engine.screen[Xs,Ys].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

Here the `engine.X` and `engine.Y` coordinates are still allowed to go beyond the screen, but the pixel is displayed such that it wraps back round. We can interpret this as the pixel moving on to another screen.

Now let's move towards giving our pixel a more exciting world to explore. We'll use a function to decide what colour each point should be. We'll start by not changing very much.

In [ ]:
import numpy as np

def get_color(X,Y):
    return 'green'

# this is the function that does everything
def next_frame (engine):
    
    # remove initial text
    if engine.controller['A'].value:
        engine.screen['text'].set_text('')
    
    # change player position
    if engine.controller['up'].value:
        engine.Y -= 1
    if engine.controller['down'].value:
        engine.Y += 1
    if engine.controller['right'].value:
        engine.X += 1
    if engine.controller['left'].value:
        engine.X -= 1
    
    # set all pixels to green
    for x in range(engine.L):
        for y in range(engine.L):
            # get the 'world' coordinates X,Y from the onscreen coordinates x,y
            X = np.floor(engine.X/engine.L)*engine.L+x
            Y = np.floor(engine.Y/engine.L)*engine.L+y
            # set it to whatever colour it should be
            engine.screen[x,y].set_color(get_color(X,Y))
            
    # draw the player pixel
    Xs = engine.X%engine.L
    Ys = engine.Y%engine.L
    engine.screen[Xs,Ys].set_color('red')
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

Now let's change `get_color` to create a beach.

In [ ]:
def get_color(X,Y):
    if X<12:
        color = 'green'
    else:
        color = 'orange'
    return color
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

We'll now put a single qubit circuit inside this terrain generator, so that we can experiment with single qubit gates by making terrain. For that we'll need some Qiskit tools.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

First, let's see what a circuit looks like. Here's one with just a `ry` gate for a given angle of rotation.

In [ ]:
theta = np.pi/2

qc = QuantumCircuit(1)
qc.ry(theta,0)

state = Statevector.from_instruction(qc)
probs = state.probabilities_dict()

print(probs)

Let's use the probability of the output '1' as a height, and set the colour accordingly.

In [ ]:
def get_color(X,Y):
    
    # set an angle for ry based on the coordinate
    thet = X*(np.pi/16)
    
    # create and simulate the circuit
    qc = QuantumCircuit(1)
    qc.ry(theta,0)
    state = Statevector.from_instruction(qc)
    probs = state.probabilities_dict()
    
    # get the prob of '1'
    if '1' in probs:
        height = probs['1']
    else:
        height = 0
    
    # set colour accordingly
    if height<0.1: # sea/river
        color = 'blue'
    elif height<0.3: # beach
        color = 'orange'
    elif height<0.9: # grass
        color = 'green'
    else: # mountain
        color = 'grey'
        
    return color
    
# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

We can build on this to create more complex terrain.

In [ ]:
import random

seed = [random.random() for _ in range(4)]

def get_color(X,Y):
    
    qc = QuantumCircuit(1)

    theta1 = (seed[0]*X+seed[1]*Y)*np.pi/16
    theta2 = (seed[2]*X-seed[3]*Y)*np.pi/16
    qc.ry(theta1,0)
    qc.rx(theta2,0)

    state = Statevector.from_instruction(qc)
    probs = state.probabilities_dict()
    
    try:
        height = probs['1']
    except:
        height = 0
        
    # set colour accordingly
    if height<0.1: # sea/river
        color = 'blue'
    elif height<0.3: # beach
        color = 'orange'
    elif height<0.9: # grass
        color = 'green'
    else: # mountain
        color = 'grey'
        
    return color

# run the game
engine = JupyterWidgetEngine(start,next_frame,L=8)

Now experiment by making your own circuits to generate more interesting terrain. You might find [this](https://javafxpert.github.io/grok-bloch/) to be a handy tool to determine what different gates will do.